In [9]:
import pandas as pd
import re

In [10]:
df = pd.read_csv("./../../data_llm_bs.csv")
df.head()

,Unnamed: 0,created_at,text,handle,displayname,like,repost,reply
0,0,2025-03-25T07:55:40.962Z,🚨Voici un outil Open Source très intéressant e...,duggytuxy.bsky.social,Laurent M.,1,0,0
1,1,2025-03-24T22:14:22.627Z,“If I had to speculate” proceeds to continue b...,ilarias.bsky.social,Ilaria 🇺🇦🇵🇱🇱🇹,1,0,1
2,2,2025-03-24T21:51:17.024Z,Modest proposal on a fix to this (𝘳𝘦𝘱𝘭𝘢𝘤𝘦 𝘵𝘩𝘦 ...,rogntudju.bsky.social,𓃰 Rogntuudju 𓃰,1,0,0
3,3,2025-03-24T20:59:01.804Z,Et je te parle même pas des groupes plus ou mo...,akbida.bsky.social,Abdellatif Kbida,1,0,1
4,4,2025-03-24T18:04:17.512Z,« les constatations humaines » : donc ils bluf...,caligans.bsky.social,phantôme,0,0,1


## Regex

In [4]:
re.findall?

Signature: re.findall(pattern, string, flags=0)
Docstring:
Return a list of all non-overlapping matches in the string.

If one or more capturing groups are present in the pattern, return
a list of groups; this will be a list of tuples if the pattern
has more than one group.

Empty matches are included in the result.
File:      /opt/anaconda3/lib/python3.11/re/__init__.py
Type:      function

In [14]:
mask = r"\b(\d{4})\b"

- on veut quatre caractères numériques à la suite : `\d{4}`
- on veut que la regex renvoie que cet élément : `(\d{4})`
- on veut que ces quatre caractères soit en bordure de part et d'autre : `\b(\d{4})\b`

In [15]:
re.findall(mask,"Ceci est une année 2342, et ceci est une autre 3323")

['2342', '3323']

In [20]:
df["text"].apply(lambda x : re.findall(mask, x))

0           []
1           []
2           []
3           []
4           []
         ...  
2394        []
2395        []
2396        []
2397    [2023]
2398        []
Name: text, Length: 2399, dtype: object

In [17]:
df["text"].str.extractall(mask)

,,0
,match,
6,0,2025
23,0,2025
40,0,2025
50,0,1300
56,0,2020
...,...,...
2329,0,2309
2339,0,2307
2384,0,3000


## Découper un texte en mots

In [22]:
texte = "il fait beau aujourd'hui"
mots = re.split(r"\W+", texte)
mots

['il', 'fait', 'beau', 'aujourd', 'hui']

## Contruire la matrice document/terme

In [25]:
#pip install scikit-learn

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
CountVectorizer

sklearn.feature_extraction.text.CountVectorizer

In [28]:
modele = CountVectorizer(min_df=5)

In [39]:
modele.fit(df["text"])

CountVectorizer(min_df=5)

In [35]:
text_tranform  = modele.transform(df["text"])

In [45]:
pd.DataFrame(text_tranform.toarray(), 
             columns= modele.get_feature_names_out())

,000,01,02,05,10,100,10x,11,12,13,...,étudiants,été,évaluation,évidemment,évident,éviter,évoluer,évolution,êtes,être
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2394,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2395,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Découverte de spacy

In [47]:
#pip install spacy

In [49]:
#!python -m spacy download fr_core_news_md

In [50]:
import spacy

In [51]:
modele = spacy.load("fr_core_news_md")

In [54]:
texte_transforme = modele("Ceci est un texte")

In [55]:
type(texte_transforme)

spacy.tokens.doc.Doc

In [60]:
mot = texte_transforme[1]

In [64]:
for token in texte_transforme:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Ceci ceci PRON PRON nsubj Xxxx True True
est être AUX AUX cop xxx True True
un un DET DET det xx True True
texte texte NOUN NOUN ROOT xxxx True False


In [65]:
def filtrer_verbes(texte):
    texte_transforme = modele(texte)
    return [mot.text for mot in texte_transforme if mot.pos_ == "NOUN"]

In [66]:
filtrer_verbes("Ceci est un texte")

['texte']

In [67]:
df["text"].apply(filtrer_verbes)

0       [outil, Open, Source, ℹ️Garak, recherche, fuit...
1       [proceeds, baselessly, paragraphs, garbanzo, m...
2                                     [𝘳𝘦𝘱𝘭𝘢𝘤𝘦, 𝘓𝘓𝘔, 𝘓𝘓𝘔]
3       [groupes, blogs, LLM, vessies, lanternes, fois...
4                          [constatations, LLM, rapports]
                              ...                        
2394           [caca, boîtes, soi, -, LLM, nuance, chose]
2395    [might, But, m, blogs, repo, concert, tools, S...
2396    [think, that’, B2B, transaction, value, But, n...
2397    [wrote, -, level, guide, novel, https://allenp...
2398                                [Pi, AI, bestie, LLM]
Name: text, Length: 2399, dtype: object

In [72]:
texte = "Vous finissez le semestre bientôt à Centrale Méditerrannée"

In [73]:
txt_t = modele(texte)

In [74]:
txt_t.ents

(Centrale Méditerrannée,)

In [75]:
df["text"].apply(lambda x: modele(x).ents)

0       ((Garak), (Vulnerability, Scanner), (ℹ️Garak),...
1       ((baselessly, speculating, as, he, did, for, p...
2                                             ((Modest),)
3                                 ((empoissonner), (IAG))
4                                                      ()
                              ...                        
2394                        ((Elon), (Twitter), (Google))
2395    ((Oreilly, books, and, trying, to, do, stuff),...
2396                 ((I), (B2B, transaction, that, has))
2397                                               ((I),)
2398                       ((my, personal, AI), (#, LLM))
Name: text, Length: 2399, dtype: object

## Utiliser Gliner pour du NER

In [77]:
#pip install gliner

In [ ]:
model = GLiNER.from_pretrained("urchade/gliner_multi_pii-v1")

In [7]:
texte = "During my lunch, I ate rice and leeks salad"
model.predict_entities(texte, ["food"])

[{'start': 23,
  'end': 43,
  'text': 'rice and leeks salad',
  'label': 'food',
  'score': 0.9316188097000122}]

In [ ]:
#pip install -U gliner

In [13]:
t = df["text"][0:500].apply(lambda x : model.predict_entities(x, ["personnes humaines"]))

## Analyse de sentiment

In [16]:
from transformers import pipeline

# Load the classification pipeline with the specified model
pipe = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

# Classify a new sentence
sentence = "I love this product! It's amazing and works perfectly."
result = pipe(sentence)

# Print the result
print(result)

config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use mps:0


[{'label': 'Very Positive', 'score': 0.5922620892524719}]


In [25]:
t = df["text"].apply(lambda x : pipe(x)[0]["label"])
t.value_counts()

Neutral          1150
Very Negative     537
Very Positive     286
Positive          238
Negative          188
Name: text, dtype: int64

In [26]:
df["sentiment"] = t

In [32]:
list(df[df["sentiment"] == "Very Negative"]["text"])[0:10]

['“If I had to speculate” proceeds to continue baselessly speculating as he did for paragraphs of complete garbanzo. I’m 100% convinced this is majority written or “parsed” through an LLM. There’s genuinely no way a functional human being with a basic capacity to check Wikipedia could write this.',
 'Et je te parle même pas des groupes plus ou moins organisés qui remplissent le web de faux blogs pour empoissonner les LLM et faire passer des vessies pour des lanternes. Quand répéter 100 fois un mensonge en fait une "vérité" pour une IAG qui le propagera au grand public 😪.',
 '«\u202fles constatations humaines\u202f» : donc ils bluffent et si ça se trouve ils utilisent un LLM pour générer les rapports ?',
 'Comment les IA transforment tout code en code open source | Intelligence artificielle | Le site de Korben\n\nhttps://korben.info/llm-decompilation-claude-code-hack-intelligence-artificielle.html\n\n',
 'Je serais quand même un peu surpris que les nigauds qui ont entraîné leurs LLM sur

## Interroger un service distant

In [35]:
import requests
import json

# Définir la prompt de classification
prompt = """Est-ce que ce texte est positif ou négatif.
Je veux uniquement une réponse en POSITIF, NEGATIF ou NEUTRE.
Le texte est : Je ne comprends rien au NLP"""

# Envoi de la requête à Ollama
response = requests.post(
    'https://llm-tools-alpha.huma-num.fr/ES_hpEC1VnleR_8C52IbSxHF/api/generate',
    json={
        'model': 'llama3.2',
        'prompt': prompt,
        'stream': False  # stream=False pour avoir une réponse simple
    }
)

# Traitement de la réponse
result = response.json()
result

{'model': 'llama3.2',
 'created_at': '2025-03-25T13:13:35.583693252Z',
 'response': 'NEGATIF.',
 'done': True,
 'done_reason': 'stop',
 'context': [128006,
  9125,
  128007,
  271,
  38766,
  1303,
  33025,
  2696,
  25,
  6790,
  220,
  2366,
  18,
  271,
  128009,
  128006,
  882,
  128007,
  271,
  14101,
  54312,
  1744,
  3846,
  69067,
  1826,
  20940,
  333,
  6033,
  308,
  19395,
  50848,
  627,
  30854,
  5320,
  2249,
  29121,
  1133,
  6316,
  90509,
  665,
  27592,
  964,
  2843,
  11,
  85165,
  835,
  2843,
  6033,
  8014,
  1406,
  793,
  627,
  2356,
  69067,
  1826,
  551,
  14465,
  841,
  60946,
  82,
  55455,
  8065,
  452,
  12852,
  128009,
  128006,
  78191,
  128007,
  271,
  98227,
  835,
  2843,
  13],
 'total_duration': 49503571,
 'load_duration': 14623277,
 'prompt_eval_count': 70,
 'prompt_eval_duration': 6407895,
 'eval_count': 5,
 'eval_duration': 27451939}